In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special

In [10]:
class NeuralNetwork:
    # initialization
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # learning rate
        self.lr = learningrate
        
        # initialization of weight martices
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        # activation function
        self.activation_function = lambda x: scipy.special.expit(x)
    
    # training
    def train(self, input_list, target_list):
        # transform input and target lists
        inputs = np.array(input_list, ndmin=2).T
        targets = np.array(target_list, ndmin=2).T
        
        # hidden layer inputs
        hidden_inputs = np.dot(self.wih, inputs)
        # hidden layer outputs
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # final (output) layer inputs
        final_inputs = np.dot(self.who, hidden_outputs)
        # final (output) layer outputrs (final results)
        final_outputs = self.activation_function(final_inputs)
        
        # error = target_value - actual_value
        output_errors = targets - final_outputs
        
        # hidden layer errors are output_errors distributed 
        # in proportion to the weighting factors of the links
        # and recombined on the hidden nodes
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # update the weights between hidden and output (final) layers
        self.who += self.lr * np.dot(output_errors * final_outputs * (1.0 - final_outputs), np.transpose(hidden_outputs))
        # update the weights between input and hidden layers
        self.wih += self.lr * np.dot(hidden_errors * hidden_outputs * (1.0 - hidden_outputs), np.transpose(inputs))
    
    # query for nodes outputs
    def query(self, input_list):
        # transform list to 2D-array to be able to perform matrix operations
        inputs = np.array(input_list, ndmin=2).T
        
        # hidden layer inputs
        hidden_inputs = np.dot(self.wih, inputs)
        # hidden layer outputs
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # final (output) layer inputs
        final_inputs = np.dot(self.who, hidden_outputs)
        # final (output) layer outputrs (final results)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs        

In [8]:
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

learning_rate = 0.3

n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [11]:
n.query([1., .5, -1.5])

array([[0.53385844],
       [0.46615744],
       [0.59705921]])